# Exercise 3

## Import packages

In [4]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

from tqdm import trange, tqdm

## Task 1
(3 points)

Implement the training loop for one training epoch.
An epoch trains on the whole training dataset once.

In [5]:
def train(model, use_cuda, train_loader, optimizer, epoch, log_interval):
    """
    Train one epoch
    
    model -- the neural network
    use_cuda -- true if GPU should be used
    train_loader -- data loader
    optimizer -- network optimizer
    epoch -- number of current epoch
    log_interval -- number of training steps between logs
    """

    criterion = F.nll_loss # negative log likelihood loss here

    # Use GPU if available
    device = torch.device("cuda" if use_cuda else "cpu")
    model.to(device)

    # TODO: set the model to train mode
    model.train()
    # TODO: enumerate over the dataloader to get mini batches
    #       of images and ground truth labels
    # HINT: the builtin python function enumerate() also gives you indices
    for batch_idx,item in enumerate(train_loader):
        # get the images and the truth labels
        images, truth_labels = item
        images, truth_labels = images.to(device), truth_labels.to(device)

        # TODO: set the optimizers gradients to zero
        optimizer.zero_grad()
        # TODO: run the network
        outputs = model(images)
        # TODO: compute negative log likelihood loss
        loss = criterion(outputs,truth_labels)
        # TODO: do backpropagation
        loss.backward()
        # TODO: optimize
        optimizer.step() # update the weights

        # TODO: print current loss for every nth ("log_interval"th) iteration
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(images)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

We already implemented the validation function for you (this is essentially validate() from the last exercise)

In [6]:
def validate(model, use_cuda, test_loader):
    """
    Compute test metrics
    
    model -- the neural network
    use_cuda -- true if GPU should be used
    test_loader -- data loader
    """
    # create a 10x10 grid of subplots
    _, axis = plt.subplots(10, 10)
    
    # set model to evaluation mode
    model.eval()
    test_loss = 0
    correct = 0
    plotted = 0
    
    # disable gradients globally
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            # for each batch
            if use_cuda:
                # transfer to GPU
                data = data.cuda()
                target = target.cuda()
            
            # run network and compute metrics
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            
            img_correct = pred.eq(target.view_as(pred))
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            # plot the first 100 images
            img_idx = 0
            data = data.cpu().numpy()
            
            #while plotted < 100 and img_idx < data.shape[0]:
            while plotted < 10 and img_idx < data.shape[0]:# i reduced the number of plots
                # compute position of ith image in the grid
                y = plotted % 10
                x = plotted // 10
                
                # convert image tensor to numpy array and normalize to [0, 1]
                img = data[img_idx, 0]
                img = (img - np.min(img)) / (np.max(img) - np.min(img))
                
                # make wrongly predicted images red
                img = np.stack([img] * 3, 2)
                if img_correct[img_idx] == 0:
                    img[:, :, 1:] = 0.0
                
                # disable axis and show image
                axis[y][x].axis('off')
                axis[y][x].imshow(img)
                
                # show the predicted class next to each image
                axis[y][x].text(30, 25, pred[img_idx].item())
                
                plotted += 1
                img_idx += 1
            
    test_loss /= len(test_loader.dataset)

    # show results
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    print("Does it work perfectly fine till this point")
    plt.show()
    print(" is the problem somewhere here")

## Task 2
(4 points)

Implement a five-layer fully connected neural network.
The dimensions (without batch size) should change like this: 784->200->100->60->30->10
Use log softmax to compute the class predictions.

Run the code at the end of the notebook to train and validate your implementation.

### Task 2.1
* sigmoid non-linear activation function
* note that the last layer does not need an activation function!

### Task 2.2
* add a new class "FCNet2"
* replace sigmoid with ReLU

### Task 2.3
* add a new class "FCNet2"
* add batch normalization to the first and third layers (note the difference between 1D/2D/3D versions)


**NOTE:** The perfomance should improve slightly with each step. However, due to the random weight initialization applied by PyTorch, your results may vary a bit between trainings.

In [7]:
class FCNet1(nn.Module):
    """
    Fully Connected Neural Network
    
    Five fully connected layers with sigmoid non-linearity
    
    Dimensions
    784->200->100->60->30->10
    """
    def __init__(self, layer_size_list=[784, 200, 100, 60, 30, 10]): # add the dropout probability here as well
        super(FCNet1, self).__init__()
        self.layer_size_list = layer_size_list
        # TODO: initialize network layers
        # HINT: take a look at "torch.nn" (imported as "nn")
        self.linear_layers = nn.ModuleList()
        for i in range(len(self.layer_size_list) - 1):
            self.linear_layers.append(
                nn.Linear(self.layer_size_list[i], self.layer_size_list[i + 1])
            )
    def forward(self, x):
        # TODO: reshape batch of images to batch of 1D vectors
        x = x.view(x.size(0), -1)# This would reshape input from (batch_size, 1, 28, 28) to (batch_size, 784)
        
        # TODO: run network layers
        for layer in self.linear_layers[:-1]:
            x = torch.sigmoid(layer(x))
        # Apply the final layer (no activation)
        x = self.linear_layers[-1](x)

        # TODO: compute log softmax over the output
        output = F.log_softmax(x, dim=1)

        return output

In [8]:
class FCNet2(nn.Module):
    def __init__(self, layer_size_list=[784, 200, 100, 60, 30, 10]):# add dropout later
        super(FCNet2, self).__init__()

        self.layer_size_list = layer_size_list
        
        self.linear_layers = nn.ModuleList() # this is list of linear layer
        for i in range(len(self.layer_size_list) - 1):
            self.linear_layers.append(
                nn.Linear(self.layer_size_list[i], self.layer_size_list[i + 1])
            )
    def forward(self, x):
        # TODO: reshape batch of images to batch of 1D vectors
        x = x.view(x.size(0), -1)# This would reshape input from (batch_size, 1, 28, 28) to (batch_size, 784)
        
        # TODO: run network layers
        for layer in self.linear_layers[:-1]:
            x = F.relu(layer(x))
        # Apply the final layer (no activation)
        x = self.linear_layers[-1](x)

        # TODO: compute log softmax over the output
        output = F.log_softmax(x, dim=1)

        return output

In [9]:
class FCNet3(nn.Module):
    def __init__(self, layer_size_list=[784, 200, 100, 60, 30, 10]):

        super(FCNet3, self).__init__()

        self.layer_size_list = layer_size_list

        self.linear_layers = nn.ModuleList() # this is list of linear layer
        self.batchnorm_layers = nn.ModuleDict() # list of BN layer
        
        for i in range(len(self.layer_size_list) - 1):
            self.linear_layers.append(
                nn.Linear(self.layer_size_list[i], self.layer_size_list[i + 1])
            )
            if i==0 or i==2:
                self.batchnorm_layers[str(i)]=nn.BatchNorm1d(self.layer_size_list[i + 1])

    def forward(self, x):
        # TODO: reshape batch of images to batch of 1D vectors
        x = x.view(x.size(0), -1)# This would reshape input from (batch_size, 1, 28, 28) to (batch_size, 784)

        # TODO: run network layers
        for i in range(len(self.linear_layers)-1):
            x=self.linear_layers[i](x)
            # apply BN layer to 1st and 3rd layer
            if i==0 or i==2:
                x=self.batchnorm_layers[str(i)](x)
            # apply activation function
            x = F.relu(x)
        # Apply the final layer (no activation)
        x = self.linear_layers[-1](x)

        # TODO: compute log softmax over the output
        output = F.log_softmax(x, dim=1)

        return output

## Task 3
(3 points)

Implement a convolutional neural network, consisting of two convolutional and two fully connected layers.
This time, the dimensions (without batch size) should change like this: 1x28x28->32x26x26->64x12x12->128->10

### Task 3.1
* two convolutional layers (kernel size 3)
* two fully-connected layers
* ReLU activation function

### Task 3.2
* add batch normalization to first convolutional and first fully connected layer

### Task 3.3
* use max pooling instead of stride to reduce the dimensions to 64x12x12

In [10]:
class ConvNet1(nn.Module):
    """
    Convolutional Neural Network
    
    Two convolutional layers and two fully connected layers
    
    Dimensions:
    1x28x28->32x26x26->64x12x12->128->10
    """
    def __init__(self):
        super(ConvNet1, self).__init__()
        
        # TODO: initialize network layers
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)  # 28x28 -> 26x26
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3) # 26x26 -> 24x24

        # Flattened size after conv2: 64 * 24 * 24 = 36864
        self.fc1 = nn.Linear(64 * 24 * 24, 128)
        self.fc2 = nn.Linear(128, 10)
        

    def forward(self, x):
        # Conv layers with ReLU
        x = F.relu(self.conv1(x))  # 1x28x28 -> 32x26x26
        x = F.relu(self.conv2(x))  # 32x26x26 -> 64x24x24

        # Flatten
        x = x.view(x.size(0), -1)  # batch x 36864

        # Fully connected layers with ReLU
        x = F.relu(self.fc1(x))    # 36864 -> 128
        x = self.fc2(x)            # 128 -> 10

        # Log softmax for class probabilities
        output = F.log_softmax(x, dim=1)
        
        return output

In [11]:
class ConvNet2(nn.Module):
    """
    Convolutional Neural Network
    
    Two convolutional layers and two fully connected layers
    Batch normalisation after 1CNN and 1FC layer
    
    Dimensions:
    1x28x28->32x26x26->64x12x12->128->10
    """
    def __init__(self):
        super(ConvNet2, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)   # 28x28 -> 26x26
        self.bn1 = nn.BatchNorm2d(32)  # BatchNorm after conv1

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)  # 26x26 -> 24x24

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 24 * 24, 128)
        self.bn_fc1 = nn.BatchNorm1d(128)  # BatchNorm after first FC
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # Conv layer 1 + BN + ReLU
        x = F.relu(self.bn1(self.conv1(x)))   # 1x28x28 -> 32x26x26
        # Conv layer 2 + ReLU
        x = F.relu(self.conv2(x))             # 32x26x26 -> 64x24x24
        # Flatten
        x = x.view(x.size(0), -1)             # batch x 36864
        # FC layer 1 + BN + ReLU
        x = F.relu(self.bn_fc1(self.fc1(x)))  # 36864 -> 128
        # FC layer 2
        x = self.fc2(x)                       # 128 -> 10
        # LogSoftmax
        output = F.log_softmax(x, dim=1)

        return output


In [12]:
class ConvNet3(nn.Module):
    def __init__(self):
        super(ConvNet3, self).__init__()

    def forward(self, x):
        pass

### Hyperparameters and DataLoader

In [13]:
# hyper parameters
batch_size = 64
test_batch_size = 1000
epochs = 2#10
lr = 1.0
gamma = 0.7
log_interval = 100

# use GPU if available
use_cuda = False#torch.cuda.is_available()
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# initialize data loaders
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])), batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, 
    transform=transforms.Compose([
        transforms.ToTensor(), 
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

### Fully connected networks

In the code below, IDK why when I keep `validate()` function, the kernel keeps on crashing on my laptop, thus meanwhile I have removed the validation function in the code below. (note that this code was in already present in the code, in the cell above, so we cannot simply comment `validate` function in the final solution.)

In [14]:
model = FCNet1()
if use_cuda:
    model = model.cuda()

# initialize optimizer and scheduler
optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

for epoch in trange(1, epochs + 1, desc="Epochs"):
    # train one epoch
    train(model, use_cuda, train_loader, optimizer, epoch, log_interval)
    
    # run on test dataset
    validate(model, use_cuda, test_loader)
    # I think there is some problem in the validate model
    scheduler.step()
    
    #torch.save(model.state_dict(), "models/mnist/checkpoint.pt")

Epochs:   0%|          | 0/2 [00:00<?, ?it/s]

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.365259
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.314620
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.302558
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.315111
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.295861
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.292243
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.325281
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.289016
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.300503
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.994141

Test set: Average loss: 1.8143, Accuracy: 2223/10000 (22.23%)

Does it work perfectly fine till this point


: 

In [12]:
model = FCNet2()
if use_cuda:
    model = model.cuda()

# initialize optimizer and scheduler
optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

for epoch in range(1, epochs + 1):
    # train one epoch
    train(model, use_cuda, train_loader, optimizer, epoch, log_interval)
    
    # run on test dataset
    # validate(model, use_cuda, test_loader) 
    scheduler.step()
    
    
    #torch.save(model.state_dict(), "models/mnist/checkpoint.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312175
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.377308
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.219918
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.090426
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.286938
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.086095
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.195589
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.296510
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.097019
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.090117
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.211579
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.128039
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.178205
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.144463
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.159606
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.182984
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.017491
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.028740
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.196418
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.043180
Train Epoch:

In [17]:
model = FCNet3()
if use_cuda:
    model = model.cuda()

# initialize optimizer and scheduler
optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

for epoch in range(1, epochs + 1):
    # train one epoch
    train(model, use_cuda, train_loader, optimizer, epoch, log_interval)
    
    # run on test dataset
    # validate(model, use_cuda, test_loader)
    scheduler.step()
    
    
    #torch.save(model.state_dict(), "models/mnist/checkpoint.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299743
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.196098
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.386714
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.313938
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.174311
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.234149
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.187599
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.060387
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.031403
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.180847
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.083403
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.030506
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.185305
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.217192
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.077417
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.048841
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.106582
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.030813
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.015723
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.037259
Train Epoch:

### Running the CNN network

In [22]:
model = ConvNet1()
if use_cuda:
    model = model.cuda()

# initialize optimizer and scheduler
optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

for epoch in trange(1, epochs + 1, desc="Epochs"):
    # train one epoch
    train(model, use_cuda, train_loader, optimizer, epoch, log_interval)
    
    # run on test dataset
    #validate(model, use_cuda, test_loader)
    # I think there is some problem in the validate model
    scheduler.step()
    
    #torch.save(model.state_dict(), "models/mnist/checkpoint.pt")

Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307529
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.159634
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.071590
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.043399
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.004755
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.030686
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.072325
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.018905
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.076791
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.019082


Epochs:  20%|██        | 1/5 [04:00<16:01, 240.29s/it]

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.027749
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.023949
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.008759
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.006387
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.003703
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.064060
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.005642
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.013176
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.010310
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.030749


Epochs:  40%|████      | 2/5 [08:02<12:04, 241.37s/it]

...

Epochs:  60%|██████    | 3/5 [12:06<08:05, 242.65s/it]

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.007639
Train Epoch: 4 [6400/60000 (11%)]	Loss: 0.004234
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.000111
Train Epoch: 4 [19200/60000 (32%)]	Loss: 0.000269
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.001611
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.000666
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.097966
Train Epoch: 4 [44800/60000 (75%)]	Loss: 0.000897
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.000751
Train Epoch: 4 [57600/60000 (96%)]	Loss: 0.000944


Epochs:  80%|████████  | 4/5 [16:04<04:00, 240.83s/it]

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.000025
Train Epoch: 5 [6400/60000 (11%)]	Loss: 0.000541
Train Epoch: 5 [12800/60000 (21%)]	Loss: 0.000760
Train Epoch: 5 [19200/60000 (32%)]	Loss: 0.043959
Train Epoch: 5 [25600/60000 (43%)]	Loss: 0.000812
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.000428
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.000218
Train Epoch: 5 [44800/60000 (75%)]	Loss: 0.000016
Train Epoch: 5 [51200/60000 (85%)]	Loss: 0.000080
Train Epoch: 5 [57600/60000 (96%)]	Loss: 0.000190


Epochs: 100%|██████████| 5/5 [20:15<00:00, 243.04s/it]


In [23]:
model = ConvNet2()
if use_cuda:
    model = model.cuda()

# initialize optimizer and scheduler
optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

for epoch in trange(1, epochs + 1, desc="Epochs"):
    # train one epoch
    train(model, use_cuda, train_loader, optimizer, epoch, log_interval)
    
    # run on test dataset
    #validate(model, use_cuda, test_loader)
    # I think there is some problem in the validate model
    scheduler.step()
    
    #torch.save(model.state_dict(), "models/mnist/checkpoint.pt")

Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.458563
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.076713
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.059905
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.106190
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.134752
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.110792
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.066470
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.024970
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.062919
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.055440


Epochs:  20%|██        | 1/5 [03:39<14:36, 219.11s/it]

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.016042
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.014723
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.069353
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.018531
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.172360
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.010169
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.012620
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.077748
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.039668
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.005474


Epochs:  40%|████      | 2/5 [07:14<10:50, 216.90s/it]

...

Epochs:  60%|██████    | 3/5 [10:42<07:05, 212.92s/it]

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.000889
Train Epoch: 4 [6400/60000 (11%)]	Loss: 0.004210
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.000577
Train Epoch: 4 [19200/60000 (32%)]	Loss: 0.057498
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.038408
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.004375
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.006013
Train Epoch: 4 [44800/60000 (75%)]	Loss: 0.002776
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.001744
Train Epoch: 4 [57600/60000 (96%)]	Loss: 0.004646


Epochs:  80%|████████  | 4/5 [14:13<03:32, 212.09s/it]

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.000812
Train Epoch: 5 [6400/60000 (11%)]	Loss: 0.000512
Train Epoch: 5 [12800/60000 (21%)]	Loss: 0.002137
Train Epoch: 5 [19200/60000 (32%)]	Loss: 0.004673
Train Epoch: 5 [25600/60000 (43%)]	Loss: 0.001835
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.000662
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.000350
Train Epoch: 5 [44800/60000 (75%)]	Loss: 0.014186
Train Epoch: 5 [51200/60000 (85%)]	Loss: 0.004614
Train Epoch: 5 [57600/60000 (96%)]	Loss: 0.018354


Epochs: 100%|██████████| 5/5 [17:43<00:00, 212.65s/it]


In [ ]:
model = ConvNet3()
if use_cuda:
    model = model.cuda()

# initialize optimizer and scheduler
optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

for epoch in trange(1, epochs + 1, desc="Epochs"):
    # train one epoch
    train(model, use_cuda, train_loader, optimizer, epoch, log_interval)
    
    # run on test dataset
    #validate(model, use_cuda, test_loader)
    # I think there is some problem in the validate model
    scheduler.step()
    
    #torch.save(model.state_dict(), "models/mnist/checkpoint.pt")